## 6.2.1 使用__call_()而不是forward()

In [50]:
import torch.nn as nn
import torch
from torch import optim

In [51]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [52]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 9,  8, 10,  6,  2,  1,  5,  0,  3]), tensor([4, 7]))

In [53]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [54]:
liner_model = nn.Linear(1, 1)
liner_model(t_un_val)

tensor([[2.8837],
        [0.5645]], grad_fn=<AddmmBackward0>)

## 6.2.2 回到线性模型

In [55]:
liner_model = nn.Linear(1, 1)
liner_model(t_un_val)

tensor([[0.4532],
        [0.1835]], grad_fn=<AddmmBackward0>)

In [56]:
liner_model.weight

Parameter containing:
tensor([[0.0782]], requires_grad=True)

In [57]:
liner_model.bias

Parameter containing:
tensor([0.0131], requires_grad=True)

In [58]:
x = torch.ones(1)
liner_model(x)

tensor([0.0913], grad_fn=<ViewBackward0>)

In [59]:
# 1. 批量输入
x = torch.ones(10, 1)
liner_model(x)

tensor([[0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913],
        [0.0913]], grad_fn=<AddmmBackward0>)

In [60]:
# 2. 优化批次
liner_model = nn.Linear(1, 1)
optimizer = optim.SGD(liner_model.parameters(), lr=1e-2)

In [61]:
liner_model.parameters()

<generator object Module.parameters at 0x7f17266e1b60>

In [62]:
list(liner_model.parameters())

[Parameter containing:
 tensor([[-0.0930]], requires_grad=True),
 Parameter containing:
 tensor([-0.2163], requires_grad=True)]

In [63]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Train loss {loss_train.item():.4f}, Val loss {loss_val.item():.4f}")

In [64]:
training_loop(3000, optimizer, liner_model, nn.MSELoss(), t_un_train, t_un_val, t_c_train, t_c_val)

Epoch 1, Train loss 233.9701, Val loss 75.3238
Epoch 100, Train loss 21.0432, Val loss 35.4752
Epoch 200, Train loss 16.9943, Val loss 26.1231
Epoch 300, Train loss 13.8389, Val loss 19.1502
Epoch 400, Train loss 11.3798, Val loss 13.9944
Epoch 500, Train loss 9.4634, Val loss 10.2222
Epoch 600, Train loss 7.9699, Val loss 7.4994
Epoch 700, Train loss 6.8060, Val loss 5.5691
Epoch 800, Train loss 5.8990, Val loss 4.2337
Epoch 900, Train loss 5.1921, Val loss 3.3424
Epoch 1000, Train loss 4.6411, Val loss 2.7795
Epoch 1100, Train loss 4.2118, Val loss 2.4572
Epoch 1200, Train loss 3.8772, Val loss 2.3087
Epoch 1300, Train loss 3.6165, Val loss 2.2836
Epoch 1400, Train loss 3.4133, Val loss 2.3441
Epoch 1500, Train loss 3.2549, Val loss 2.4619
Epoch 1600, Train loss 3.1315, Val loss 2.6161
Epoch 1700, Train loss 3.0353, Val loss 2.7913
Epoch 1800, Train loss 2.9603, Val loss 2.9764
Epoch 1900, Train loss 2.9019, Val loss 3.1636
Epoch 2000, Train loss 2.8564, Val loss 3.3474
Epoch 2100, T